# Recency, Frequency, Monetary Value (RFM) Analysis

You will learn about customer segments. Specifically, you will get exposure to recency, frequency and monetary value, create customer segments based on these concepts, and analyze your results.

- Recency (R)
    - How recent was each customer's last purchase
- Frequency (F)
    - How many purchases the customer has done in the last 12 months
- Monetary Value (M)
    - How much the customer has spent in the last 12 months

**Grouping RFM values**
- percentiles
- Pareto 80/20 split
- Custom - based on business knowledge

**Process of calculating percentiles**
1. Sort customers based on the metric
2. Break customers into a pre-defined number of groups of equal size
3. Assign a label to each group

**Calculate percentiles with Python**

```python
spend_quartiles = pd.qcut(data['Spend'], q = 4, labels = range(1,5))
data['Spend_Quartile'] = spend_quartiles
data.sort_values('Spend')

#Create String labels
r_labels = ['Active', 'Lapsed', 'Inactive', 'Churned']
```

In [1]:
import pandas as pd

d = {'CustomerID': [0,1,2,3,4,5,6,7],
        'Spend': [137,335,172,355,303,233,244,229]}

data = pd.DataFrame(d, columns = ['CustomerID', 'Spend'])

data

CustomerID  Spend
0           0    137
1           1    335
2           2    172
3           3    355
4           4    303
5           5    233
6           6    244
7           7    229

In [2]:
spend_quartile = pd.qcut(data['Spend'], q = 4, labels = range(1,5))

data['Spend_Quartile'] = spend_quartile

print(data.sort_values('Spend'))

   CustomerID  Spend Spend_Quartile
0           0    137              1
2           2    172              1
7           7    229              2
5           5    233              2
6           6    244              3
4           4    303              3
1           1    335              4
3           3    355              4


In [5]:
data = pd.DataFrame(columns = ['CustomerID', 'Recency_Days'], data = [[0,37], [1,235], [2, 396], [3, 72], [4, 255], [5, 393], [6, 203], [7, 133]])

data

CustomerID  Recency_Days
0           0            37
1           1           235
2           2           396
3           3            72
4           4           255
5           5           393
6           6           203
7           7           133

In [7]:
#Store labels from 4 to 1 in a decresing order
r_labels = list(range(4, 0, -1))

recency_quartiles = pd.qcut(data['Recency_Days'], q = 4, labels = r_labels)

data['Recency_Quartile'] = recency_quartiles

print(data.sort_values('Recency_Days'))

   CustomerID  Recency_Days Recency_Quartile
0           0            37                4
3           3            72                4
7           7           133                3
6           6           203                3
1           1           235                2
4           4           255                2
5           5           393                1
2           2           396                1


In [20]:
online = pd.read_csv('./datasets/chapter_2/online12M.csv', parse_dates = ['InvoiceDate'])
online['TotalSum'] = online['Quantity'] * online['UnitPrice']
online.head()

Unnamed: 0  InvoiceNo StockCode                      Description  Quantity  \
0      416792     572558     22745       POPPY'S PLAYHOUSE BEDROOM          6   
1      482904     577485     23196    VINTAGE LEAF MAGNETIC NOTEPAD         1   
2      263743     560034     23299     FOOD COVER WITH BEADS SET 2          6   
3      495549     578307    72349B  SET/6 PURPLE BUTTERFLY T-LIGHTS         1   
4      204384     554656     21756         BATH BUILDING BLOCK WORD         3   

  InvoiceDate  UnitPrice  CustomerID         Country  TotalSum  
0  2011-10-25       2.10       14286  United Kingdom     12.60  
1  2011-11-20       1.45       16360  United Kingdom      1.45  
2  2011-07-14       3.75       13933  United Kingdom     22.50  
3  2011-11-23       2.10       17290  United Kingdom      2.10  
4  2011-05-25       5.95       17663  United Kingdom     17.85

In [17]:
print('Min:{}; Max:{}'.format(min(online.InvoiceDate), max(online.InvoiceDate)))

Min:2010-12-10 00:00:00; Max:2011-12-09 00:00:00


In [18]:
import datetime
snapshot_date = max(online.InvoiceDate) + datetime.timedelta(days = 1)
snapshot_date

Timestamp('2011-12-10 00:00:00')

In [22]:
#Aggregate data on a customer level
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})

datamart.rename(columns = {'InvoiceDate': 'Recency',
                           'InvoiceNo': 'Frequency',
                           'TotalSum': 'MonetaryValue'}, inplace = True)

datamart.head()

Recency  Frequency  MonetaryValue
CustomerID                                   
12747             3         25         948.70
12748             1        888        7046.16
12749             4         37         813.45
12820             4         17         268.02
12822            71          9         146.15

## Buildng RFM Segments



In [23]:
# The recency is better when it's lower
r_labels = range(4, 0, -1)

r_quartiles = pd.qcut(datamart['Recency'], 4, labels = r_labels)
datamart = datamart.assign(R = r_quartiles.values)

datamart.head()

Recency  Frequency  MonetaryValue  R
CustomerID                                      
12747             3         25         948.70  4
12748             1        888        7046.16  4
12749             4         37         813.45  4
12820             4         17         268.02  4
12822            71          9         146.15  2

In [25]:
# The frequency and monetary values are better when they are higher
f_labels = range(1, 5)
m_labels = range(1, 5)

f_quartiles = pd.qcut(datamart['Frequency'], 4, labels = f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], 4, labels = m_labels)

datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)

datamart.head()

Recency  Frequency  MonetaryValue  R  F  M
CustomerID                                            
12747             3         25         948.70  4  4  4
12748             1        888        7046.16  4  4  4
12749             4         37         813.45  4  4  4
12820             4         17         268.02  4  3  3
12822            71          9         146.15  2  2  3

**Build RFM Segment and RFM Score**

- Concatenate RFM quartile values to RFM_Segment
- Sum RFM quartiles vales to RFM_Score

In [29]:
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])

datamart['RFM_Segment'] = datamart.apply(join_rfm, axis = 1)
datamart['RFM_Score'] =datamart[['R', 'F', 'M']].sum(axis = 1)

datamart

Recency  Frequency  MonetaryValue  R  F  M RFM_Sgment  RFM_Score  \
CustomerID                                                                     
12747             3         25         948.70  4  4  4        444       12.0   
12748             1        888        7046.16  4  4  4        444       12.0   
12749             4         37         813.45  4  4  4        444       12.0   
12820             4         17         268.02  4  3  3        433       10.0   
12822            71          9         146.15  2  2  3        223        7.0   
...             ...        ...            ... .. .. ..        ...        ...   
18280           278          2          38.70  1  1  1        111        3.0   
18281           181          2          31.80  1  1  1        111        3.0   
18282             8          2          30.70  4  1  1        411        6.0   
18283             4        152         432.93  4  4  4        444       12.0   
18287            43         15         395.76  3  3  4        334       10.0   

           RFM_Segment  
CustomerID              
12747              444  
12748              444  
12749              444  
12820              433  
12822              223  
...                ...  
18280              111  
18281              111  
18282              411  
18283              444  
18287              334  

[3643 rows x 9 columns]

**Analyzing RFM Segments**

In [33]:
datamart.groupby('RFM_Segment').size().sort_values(ascending = False)[:10]

RFM_Segment
444    372
111    345
211    169
344    156
233    129
222    128
333    120
122    117
311    114
433    113
dtype: int64

In [36]:
datamart[datamart['RFM_Segment'] == '144']

Recency  Frequency  MonetaryValue  R  F  M RFM_Sgment  RFM_Score  \
CustomerID                                                                     
12843           160         22         385.55  1  4  4        144        9.0   
13093           276         28        1185.84  1  4  4        144        9.0   
13952           218         32         722.06  1  4  4        144        9.0   
14461           149         26         340.42  1  4  4        144        9.0   
15146           165         26         364.42  1  4  4        144        9.0   
15235           218         25         381.69  1  4  4        144        9.0   
15379           170         45         867.52  1  4  4        144        9.0   
15704           141         26         462.36  1  4  4        144        9.0   
15808           307         25         441.25  1  4  4        144        9.0   
16919           157         67         534.44  1  4  4        144        9.0   
17337           152        114         413.46  1  4  4        144        9.0   
17504           207         26         677.68  1  4  4        144        9.0   
18231           193         22         444.70  1  4  4        144        9.0   
18260           173         30         571.89  1  4  4        144        9.0   

           RFM_Segment  
CustomerID              
12843              144  
13093              144  
13952              144  
14461              144  
15146              144  
15235              144  
15379              144  
15704              144  
15808              144  
16919              144  
17337              144  
17504              144  
18231              144  
18260              144

In [39]:
datamart.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count', 'sum']
}).round(1)


Recency Frequency MonetaryValue                
             mean      mean          mean count       sum
RFM_Score                                                
3.0         246.9       2.1          28.4   345    9803.3
4.0         162.2       3.1          47.8   337   16105.5
5.0         138.9       4.3          78.2   393   30746.4
6.0         101.0       6.3         146.3   444   64959.2
7.0          78.0       8.5         160.2   382   61186.3
8.0          62.6      12.8         196.3   376   73803.3
9.0          46.8      16.7         330.3   345  113943.1
10.0         31.9      24.0         443.1   355  157288.9
11.0         21.8      38.9         705.3   294  207358.9
12.0          8.0      75.6        1653.9   372  615244.9

**Grouping into named segments**

Use RFM scoreto group customers into *Gold*, *Silver*, *Bronze* segments


In [40]:
def segment_me(df):
    if df['RFM_Score'] >= 9:
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'

In [41]:
datamart['General_Segment'] = datamart.apply(segment_me, axis = 1)

datamart.groupby('General_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            205.0       2.6          38.0   682
Gold               27.0      39.4         800.8  1366
Silver             95.8       7.9         144.6  1595